## Solution for Identifying Issues in Data

#### Two Step Approach
<img src="img/data_issue_flow.png" alt="Train & Test Methodology" width="700" height="600">

## Explanation of Approach

- __Step1: Heuristic Approach__: validate each column in the data for issues pertaining "Empty Cells" and "Data Type" with the rules given in "info-field sheet". Store indices of rows of columns not meeting the condition.

- __Step2: Frequency Based Issue Identification__: All categorical columns post validation from pre defined conditions may still include erroneous values. Such values are likely to have extremly small frequency. __Assuming__ such erroneous values may have frequency < 0.1 then such values can be tagged as __likely__data issues.

- __Combining Step1 & 2:__ As an ensemble approach we can consider data issues that were predicted in both the above approaches of either. I have opted to go with selecting index of data issues per column where from approach that identifies less data issues. __Assumption__ : Anomaly or outlier in data is generally the value which is rare in the given population.

## To the Code.......

### Load requried packages

In [ ]:
import pandas as pd
import numpy as np
import dateparser
import datetime
import json

### Import data
- df is the raw data.
- metaFile is the sheet containing conditions for validation of data

In [ ]:
df = pd.read_excel("Data_Science_Problem-external.xlsx",sheet_name="Data")
metaFile = pd.read_excel("Data_Science_Problem-external.xlsx",sheet_name= "field-info")

### Create Mapping Table

- Includes Regular Expression for validation of special data types

In [ ]:
## Mapping table
mapping_dict = dict(ip4 = '[0-9]+(?:\.[0-9]+){3}', Hash = '([a-fA-F\d]{32})',
                   timepStamp = '[0-3][0-9]\s\w+\,\s[0-9]{4}\s\([0-2][0-9]:[0-5][0-9]:[0-5][0-9]\)',
                    alphaNumeric = "(?=.*[a-zA-Z])(?=.*[@#\d])[a-zA-Z\d@#]+",
                   url = '\w+\.com')

### Step1: Heuristic Approach

In [ ]:
def rule_based_data_issues(df, metaFile,mapper):
    
    df_new = df.dropna(how='all')
    # renaming of columns
    df_new = df.iloc[:,0:34]
    
    temp = [] # empty list to hold field ids of issue related fields
    for column in df_new.columns:
        temp_values = metaFile.loc[metaFile["Field Name"].str.contains(column+"$")]
        temp_values = temp_values.head(n=1)
        #print(temp_values)
        # Rules to check the data type
        temp_na = []
        temp_type = []
        # Handle rule for empty cells
        if temp_values["Can be empty?"].values == "N":
            temp_na.extend(df_new[df_new[column].isnull()].index.tolist())
        
        # Handle rules for data type
        if temp_values["Type"].values == "alpha-numeric":
            alnum = df_new[[column]].apply(lambda x : x.str.contains(mapper["alphaNumeric"]))
            temp_type.extend(alnum.index[alnum[column] == False].tolist())
#             temp_type.extend(df_new[df_new[column].str.isalnum() == True].index.tolist())  
        
        elif temp_values["Type"].values == "char string":
            temp_type.extend(df_new[df_new[column].str.isalnum()== False].index.tolist())
        
        elif temp_values["Type"].values == "Integer":
            temp_type.extend(df_new[df_new[column].str.isnumeric()== False].index.tolist())    
        
        elif temp_values["Type"].values == "number":
            temp_type.extend(df_new[df_new[column].str.isnumeric()== False].index.tolist())
        
        elif temp_values["Type"].values == "string":
            temp_type.extend(df_new[df_new[column].str.isalpha()== False].index.tolist())
        
        elif temp_values["Type"].values == "IP v4":
            ip4 = df_new[[column]].apply(lambda x : x.str.contains(mapper["ip4"]))
            temp_type.extend(ip4.index[ip4[column] == False].tolist())
        
        elif temp_values["Type"].values == "MD5 hash":
            Hash = df_new[[column]].apply(lambda x : x.str.contains(mapper["Hash"]))
            temp_type.extend(Hash.index[Hash[column] == False].tolist())
        
        elif temp_values["Type"].values == "time stamp":
            timeStamp = df_new[column].apply(lambda x : isinstance(dateparser.parse(str(x)),datetime.datetime))
            temp_type.extend(timeStamp.index[timeStamp == False].tolist())
        
        else:
            url = df_new[[column]].apply(lambda x : x.str.contains(mapper["url"]))
            temp_type.extend(url.index[url[column] == False].tolist())
            
            
        temp_column_dictionary = {column:{"Empty_Cell_Issue":temp_na , 
                                          "Type_Issue":temp_type}}
        print("Issues Identified for %s.............."%column)
        print("Blank cell issues %s"%(len(temp_na)))
        print("Data Type issues %s\n"%len(temp_type))
        #print(temp_type)
        temp.append(temp_column_dictionary)
        
    return temp

In [ ]:
issues = rule_based_data_issues(df,metaFile,mapping_dict)

### Step2: Frequency Based Approach

In [ ]:
def freq_based_issues(df, metaFile,mapper,threshold = 0.1):
    df_new = df.dropna(how='all')
    # renaming of columns
    df_new = df.iloc[:,0:34]
    
    temp = [] # empty list to hold field ids of issue related fields
    exclusded_lables = ["number","Integer","IP v4","MD5 hash","time stamp"]
    for column in df_new.columns:
        temp_values = metaFile.loc[metaFile["Field Name"].str.contains(column+"$")]
        temp_values = temp_values.head(n=1)
        #print(temp_values)
        # Rules to check the data type
        temp_frequency = []
        
        # Handle rules for data type
        if temp_values["Type"].values not in exclusded_lables:
            label_freq = df_new[[column]].apply(lambda x: x.value_counts(normalize = True))
            issue_labels = label_freq.index[label_freq[column] < threshold].tolist()
            issue_index = [df_new.index[df[column] == x].tolist()[0] for x in issue_labels]
            temp_frequency.extend(issue_index)
        temp_column_dictionary = {column:{"freq_Issue":temp_frequency}}
        temp.append(temp_column_dictionary)
        
        print("Issues Identified for %s.............."%column)
        print("Frequency based issues %s\n"%len(temp_frequency))

    return temp

In [ ]:
issues_freq = freq_based_issues(df,metaFile,mapping_dict)

### Step3: Comparing the Above Approaches to Identify final row index for data issues

- The method also serializes the identified row indices to a json file.

In [3]:
def data_issue_extractor(df, metaFile,mapper, json_serialize = True):
    
    df_new = df.dropna(how='all')
    # renaming of columns
    df_new = df.iloc[:,0:34]
    
    temp_frequency = []
    exclusded_lables = ["number","Integer","IP v4","MD5 hash","time stamp"]
    for i,column in enumerate(df_new.columns):
        temp_values = metaFile.loc[metaFile["Field Name"].str.contains(column+"$")]
        temp_values = temp_values.head(n=1)
        #print(temp_values)
        # Final rule to validate data record anomaly
        if len(issues_freq[i][column]["freq_Issue"] ) > len(issues[i][column]["Type_Issue"]):
            final_dict = {column:{"Empty_Issues_Index":issues[i][column]["Empty_Cell_Issue"],
                                 "Anomalous_Value_Index":issues_freq[i][column]["freq_Issue"]}}

        else:
            final_dict = {column:{"Empty_Issues_Index":issues[i][column]["Empty_Cell_Issue"], 
                                "Anomalous_Value_Index":issues[i][column]["Type_Issue"]}}
        temp_frequency.append(final_dict)
    
    print(temp_frequency)
    
    # Seralize object to json file
    if json_serialize:
        with open('data_issues.json', 'w') as outfile:
            json.dump(temp_frequency, outfile)

    return temp_frequency


In [ ]:
data_issues = data_issue_extractor(df,metaFile,mapping_dict,json_serialize = True)

### Wrapping up the Exploration...

- Identification of issues in data can be enhanced with basic knowledge transfer from SME.
- In the light of limited knowledge alongwtih the pre defined rules, statistical method should be incorporated to identify rare values. Rare terms are likely to be outliers and with larger size of data can be effective in identification of issues.